<a href="https://colab.research.google.com/github/dudungE/house_debt_classification_model/blob/master/%EB%B6%80%EC%8B%A4%EA%B0%80%EA%B3%84ANN(sigmoid).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [92]:
import pandas as pd
%cd /content/drive/MyDrive/부실가계 분류 모형

/content/drive/MyDrive/부실가계 분류 모형


In [119]:
df = pd.read_csv('./data/2020가계부채/data_fin.csv', engine='python')
# df=df.drop(columns='Unnamed: 0')
len(df.columns)
df

,Unnamed: 0,담보대출_대출용도_거주주택 구입,담보대출_대출용도_거주주택이외 부동산 구입,담보대출_대출용도_전(월)세 보증금 마련,담보대출_대출용도_사업자금 마련,담보대출_대출용도_생활비 마련,담보대출_대출기관_은행(2010년은 농수협 중앙회 제외),담보대출_대출기관_저축은행,담보대출_대출기관_비은행금융기관(2010년은 농수협중앙회 포함),신용대출_대출용도_거주주택 구입,신용대출_대출용도_거주주택이외 부동산 구입,신용대출_대출용도_전(월)세 보증금 마련,신용대출_대출용도_사업자금 마련,신용대출_대출용도_생활비 마련,신용대출_대출기관_은행(2010년은 농수협 중앙회 제외),신용대출_대출기관_저축은행,신용대출_대출기관_비은행금융기관(2010년은 농수협중앙회 포함),자산,부채,순자산액,가구원수,가구주 만나이,가구주 성별,가구주교육정도 통합별,가구주 혼인상태,가구주 은퇴여부,입주형태,수도권여부,가구주 직업(대분류),가구주 종사상지위,부실가계
0,0,0,0.0,10000,0.0,0,10000.0,0,0.0,0,0.0,0,6000,0,6000.0,0,0,112000,54500.0,57500.0,3,34,0,1,1,1,0,1,0,1,0
1,1,0,0.0,0,9500.0,0,9500.0,0,0.0,0,0.0,0,2000,0,0.0,2000,0,42500,17500.0,25000.0,2,45,1,1,1,1,0,1,0,1,0
2,2,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0,0,0.0,0,0,5712,0.0,5712.0,1,73,1,0,0,1,0,1,0,0,0
3,3,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0,0,0.0,0,0,14870,0.0,14870.0,2,58,0,0,1,1,0,1,0,1,0
4,4,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0,0,0.0,0,0,814,0.0,814.0,1,27,1,0,0,1,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18059,18059,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0,1000,0.0,0,0,6705,2200.0,4505.0,3,42,0,1,1,1,0,0,0,0,0
18060,18060,22000,0.0,0,0.0,0,7000.0,15000,0.0,0,0.0,0,0,0,0.0,0,0,130180,22000.0,108180.0,5,49,0,1,1,1,1,0,0,1,1
18061,18061,8000,0.0,0,0.0,0,8000.0,0,0.0,0,0.0,0,0,0,0.0,0,0,265226,29000.0,236226.0,5,57,0,1,1,0,1,0,0,0,1
18062,18062,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0,0,0.0,0,0,109695,25000.0,84695.0,2,53,1,1,0,1,0,0,0,1,0


In [120]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline

In [121]:
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

X = df.iloc[:,1:-1]
Y = df.iloc[:,-1]

# 학습과 테스트 데이터 셋으로 분리
X_train , X_test , y_train , y_test = train_test_split(X, Y,
                                                       test_size=0.3,  random_state=369, stratify=Y)

y_train

6453     0
15606    0
5520     0
4535     0
6280     0
        ..
7315     1
14104    0
347      0
14146    0
16792    0
Name: 부실가계, Length: 12644, dtype: int64

In [122]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12644 entries, 6453 to 16792
Data columns (total 29 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   담보대출_대출용도_거주주택 구입                    12644 non-null  int64  
 1   담보대출_대출용도_거주주택이외 부동산 구입              12644 non-null  float64
 2   담보대출_대출용도_전(월)세 보증금 마련               12644 non-null  int64  
 3   담보대출_대출용도_사업자금 마련                    12644 non-null  float64
 4   담보대출_대출용도_생활비 마련                     12644 non-null  int64  
 5   담보대출_대출기관_은행(2010년은 농수협 중앙회 제외)      12644 non-null  float64
 6   담보대출_대출기관_저축은행                       12644 non-null  int64  
 7   담보대출_대출기관_비은행금융기관(2010년은 농수협중앙회 포함)  12644 non-null  float64
 8   신용대출_대출용도_거주주택 구입                    12644 non-null  int64  
 9   신용대출_대출용도_거주주택이외 부동산 구입              12644 non-null  float64
 10  신용대출_대출용도_전(월)세 보증금 마련               12644 non-null  int64  
 11  신용대출_대출용도_사업자금 마련        

In [123]:
print('학습 데이터 레이블 값 비율')
print(y_train.value_counts()/y_train.shape[0] * 100)
print('테스트 데이터 레이블 값 비율')
print(y_test.value_counts()/y_test.shape[0] * 100)

학습 데이터 레이블 값 비율
0    92.280924
1     7.719076
Name: 부실가계, dtype: float64
테스트 데이터 레이블 값 비율
0    92.269373
1     7.730627
Name: 부실가계, dtype: float64


In [124]:
# MinMaxScaler
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [125]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state=369)
X_train_over, y_train_over = smote.fit_sample(X_train, y_train)
print('SMOTE 적용 전 학습용 피처/레이블 데이터 세트: ', X_train.shape, y_train.shape)
print('SMOTE 적용 후 학습용 피처/레이블 데이터 세트: ', X_train_over.shape, y_train_over.shape)
print('SMOTE 적용 후 레이블 값 분포: \n', pd.Series(y_train_over).value_counts())

SMOTE 적용 전 학습용 피처/레이블 데이터 세트:  (12644, 29) (12644,)
SMOTE 적용 후 학습용 피처/레이블 데이터 세트:  (23336, 29) (23336,)
SMOTE 적용 후 레이블 값 분포: 
 1    11668
0    11668
dtype: int64


In [126]:
# 딥러닝을 구동하는 데 필요한 케라스 함수를 불러옵니다.
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# 필요한 라이브러리를 불러옵니다.
import numpy
import tensorflow as tf

# 실행할 때마다 같은 결과를 출력하기 위해 설정하는 부분입니다.
numpy.random.seed(3)
tf.random.set_seed(3)


# 모델을 설정합니다.
model = Sequential()
model.add(Dense(20, input_dim=29, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.summary()
# 모델을 컴파일합니다.
model.compile(loss='binary_crossentropy',
             optimizer='adam',
             metrics=['accuracy'])
# 모델을 실행합니다.
model.fit(X_train_over, y_train_over, epochs=8, batch_size=10, validation_split=0.2)

# 결과를 출력합니다.
print("\n Accuracy: %.4f" % (model.evaluate(X, Y)[1]))

Model: "sequential_26"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_69 (Dense)             (None, 20)                600       
_________________________________________________________________
dense_70 (Dense)             (None, 8)                 168       
_________________________________________________________________
dense_71 (Dense)             (None, 1)                 9         
Total params: 777
Trainable params: 777
Non-trainable params: 0
_________________________________________________________________
Epoch 1/8
1867/1867 [==============================] - 4s 2ms/step - loss: 0.5533 - accuracy: 0.7177 - val_loss: 0.5564 - val_accuracy: 0.6902
Epoch 2/8
1867/1867 [==============================] - 3s 1ms/step - loss: 0.4339 - accuracy: 0.8029 - val_loss: 0.4935 - val_accuracy: 0.7796
Epoch 3/8
1867/1867 [==============================] - 3s 2ms/step - loss: 0.3979 - accuracy: 0.817

In [127]:
# 최종결과값으로 반환하기(0,1)

y_predict = model.predict(X_test)    # 소수점?
y_predict

result=[]

for i in y_predict:
  if i >0.5:
    result.append(1)
  else:
    result.append(0)

# 혼동행렬 출력

labels = y_test
predictions = result


cfm = tf.math.confusion_matrix(
    labels, predictions, num_classes=None, weights=None, dtype=tf.dtypes.int32,
    name=None
)

# 혼동행렬에서 성능지표 계산 구현

cfm
n11=cfm[0][0]
n12=cfm[0][1]
n22=cfm[1][1]
n21=cfm[1][0]

print("accuracy:",(n11+n22)/(n11+n12+n22+n21))
print("prediction:",(n22)/(n22+n12))
print("recall:",(n22)/(n22+n21))

accuracy: tf.Tensor(0.8051660516605166, shape=(), dtype=float64)
prediction: tf.Tensor(0.25998492840994725, shape=(), dtype=float64)
recall: tf.Tensor(0.8233890214797136, shape=(), dtype=float64)


### 성능지표 일괄 출력

In [128]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import roc_auc_score

# get_clf_eval 함수로 성능지표들 한눈에 보기

def get_clf_eval(y_test, pred):
    confusion = confusion_matrix( y_test, pred)
    accuracy = accuracy_score(y_test , pred)
    precision = precision_score(y_test , pred)
    recall = recall_score(y_test , pred)
    f1 = f1_score(y_test,pred)
    # ROC-AUC 추가 
    roc_auc = roc_auc_score(y_test, pred)
    print('오차 행렬')
    print(confusion)
    # ROC-AUC print 추가
    print('정확도: {0:.4f}, 정밀도: {1:.4f}, 재현율: {2:.4f},\
    F1: {3:.4f}, AUC:{4:.4f}'.format(accuracy, precision, recall, f1, roc_auc))
    
    
# get_model_train_eval 함수로 모델 학습과 성능지표(get_clf_eval)한번에 해결
def get_model_train_eval(model, ftr_train=None, ftr_test=None, tgt_train=None, tgt_test=None):
    model.fit(ftr_train, tgt_train)
    pred = model.predict(ftr_test)
    get_clf_eval(tgt_test, pred)    

In [129]:
 from sklearn.metrics import roc_auc_score
 
 roc_auc_score(labels, predictions)

 get_clf_eval(labels,predictions)

오차 행렬
[[4019  982]
 [  74  345]]
정확도: 0.8052, 정밀도: 0.2600, 재현율: 0.8234,    F1: 0.3952, AUC:0.8135
